# Code for the paper: 

## Efficient and Mobile Deep Learning Architectures for Fast Identification of BacterialStrains in Resource-Constrained Devices

### Architecture: EfficientNet B1
### Data: Augmented + Cross-Validation

#### Loading libraries

In [1]:
# Imports here
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch.nn.functional as F
import pandas as pd
import time
import numpy as np

from PIL import Image
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import KFold
from torch import nn
from torch import optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms, models
from pytorch_model_summary import summary

# Archs not in Pytorch
from efficientnet_pytorch import EfficientNet

# External functions
from scripts.utils import *

In [2]:
print(torch.__version__)

import sys
print(sys.version)

1.5.1
3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]


## Data paths and hyperparameters

#### Hyperparameters and dataset details. 

In [3]:
# Dataset details
dataset_version = 'original' # original or augmented
img_shape = (224,224)
img_size = str(img_shape[0])+"x"+str(img_shape[1])

# Root directory of dataset
data_dir = '/home/yibbtstll/venvs/pytorch_gpu/CySDeepBacterial/Dataset/DIBaS_augmented/'

train_batch_size = 32
val_test_batch_size = 32
feature_extract = False
pretrained = True
h_epochs = 10
kfolds = 10

## Data preparation and loading

#### Defining transforms and creating dataloaders

In [4]:
# Define transforms for input data
training_transforms = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
total_set = datasets.ImageFolder(data_dir, transform=training_transforms)

# Defining folds
splits = KFold(n_splits = kfolds, shuffle = True, random_state = 42)

#### Visualizing the target classes in dataset

In [5]:
train_labels = {value : key for (key, value) in total_set.class_to_idx.items()}
    
print(len(train_labels)) 
print(train_labels)

32
{0: 'Acinetobacter.baumanii', 1: 'Actinomyces.israeli', 2: 'Bacteroides.fragilis', 3: 'Bifidobacterium.spp', 4: 'Clostridium.perfringens', 5: 'Enterococcus.faecalis', 6: 'Enterococcus.faecium', 7: 'Escherichia.coli', 8: 'Fusobacterium', 9: 'Lactobacillus.casei', 10: 'Lactobacillus.crispatus', 11: 'Lactobacillus.delbrueckii', 12: 'Lactobacillus.gasseri', 13: 'Lactobacillus.jehnsenii', 14: 'Lactobacillus.johnsonii', 15: 'Lactobacillus.paracasei', 16: 'Lactobacillus.plantarum', 17: 'Lactobacillus.reuteri', 18: 'Lactobacillus.rhamnosus', 19: 'Lactobacillus.salivarius', 20: 'Listeria.monocytogenes', 21: 'Micrococcus.spp', 22: 'Neisseria.gonorrhoeae', 23: 'Porfyromonas.gingivalis', 24: 'Propionibacterium.acnes', 25: 'Proteus', 26: 'Pseudomonas.aeruginosa', 27: 'Staphylococcus.aureus', 28: 'Staphylococcus.epidermidis', 29: 'Staphylococcus.saprophiticus', 30: 'Streptococcus.agalactiae', 31: 'Veionella'}


## Model definition and inicialization

#### Freezing pre-trained parameters, finetunning the classifier to output 32 classes.

In [6]:
# Freeze pretrained model parameters to avoid backpropogating through them
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Setting grad to false.")
        for param in model.parameters():
            param.requires_grad = False
    
    return model

def get_device():
    # Model and criterion to GPU
    if torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b1')
model

Loaded pretrained weights for efficientnet-b1


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [8]:
def load_model():
    # Transfer Learning
    model = EfficientNet.from_pretrained('efficientnet-b1')
    
    # Mode
    model = set_parameter_requires_grad(model, feature_extract)
    
    # Fine tuning
    # Build custom classifier
    model._fc = nn.Linear(in_features=1280,
                        out_features=32)
    return model.to(get_device())

def create_optimizer(model):
    print("Running on device:", get_device())
    # Parameters to update
    params_to_update = model.parameters()

    if feature_extract:
        params_to_update = []
        for param in model.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    else:
        n_params = 0
        for param in model.parameters():
            if param.requires_grad == True:
                n_params += 1


    # Loss function and gradient descent

    criterion = nn.CrossEntropyLoss().to(get_device())

    optimizer = optim.Adam(params_to_update, 
                          lr=0.0001, 
                          weight_decay=0.000004)
    
    return criterion, model, optimizer

## Training, validation and test functions

In [9]:
# Variables to store fold scores
train_acc = []
test_top1_acc = []
test_top5_acc = []
test_precision = []
test_recall = []
test_f1 = []
times = []

for fold, (train_idx, valid_idx) in enumerate(splits.split(total_set)):
    
    start_time = time.time()
    
    print('Fold : {}'.format(fold))
    
    # Train and val samplers
    train_sampler = SubsetRandomSampler(train_idx)
    print("Samples in training:", len(train_sampler))
    valid_sampler = SubsetRandomSampler(valid_idx)
    print("Samples in test:", len(valid_sampler))
    
    # Train and val loaders
    train_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=train_batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=1, sampler=valid_sampler)
    
    device = get_device()
    
    criterion, model, optimizer = create_optimizer(load_model())
    
    # Training
    for epoch in range(h_epochs):
        
        model.train()
        running_loss = 0.0
        running_corrects = 0
        trunning_corrects = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum()
            trunning_corrects += preds.size(0)
            

        epoch_loss = running_loss / trunning_corrects
        epoch_acc = (running_corrects.double()*100) / trunning_corrects
        train_acc.append(epoch_acc.item())
        
        print('\t\t Training: Epoch({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        
        # Validation
        
        model.eval()  
        
        vrunning_loss = 0.0
        vrunning_corrects = 0
        num_samples = 0
        
        for data, labels in valid_loader:
            
            data = data.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.no_grad():
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
            vrunning_loss += loss.item() * data.size(0)
            vrunning_corrects += (preds == labels).sum()
            num_samples += preds.size(0)
            
        vepoch_loss = vrunning_loss/num_samples
        vepoch_acc = (vrunning_corrects.double() * 100)/num_samples
        
        print('\t\t Validation({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, vepoch_loss, vepoch_acc))
    
    # Calculating and appending scores to this fold
    model.class_to_idx = total_set.class_to_idx
    scores = get_scores(model, valid_loader)
    
    test_top1_acc.append(scores[0])
    test_top5_acc.append(scores[1])
    test_precision.append(scores[2])
    test_recall.append(scores[3])
    test_f1.append(scores[4])
    
    time_fold = time.time() - start_time
    times.append(time_fold)
    print("Total time per fold: %s seconds." %(time_fold))

Fold : 0
Samples in training: 21665
Samples in test: 2408
Loaded pretrained weights for efficientnet-b1
Running on device: cuda
		 Training: Epoch(0) - Loss: 1.0642, Acc: 74.0780
		 Validation(0) - Loss: 0.2388, Acc: 92.9402
		 Training: Epoch(1) - Loss: 0.2784, Acc: 91.4563
		 Validation(1) - Loss: 0.1638, Acc: 94.9751
		 Training: Epoch(2) - Loss: 0.1846, Acc: 94.3826
		 Validation(2) - Loss: 0.1177, Acc: 95.8472
		 Training: Epoch(3) - Loss: 0.1439, Acc: 95.5504
		 Validation(3) - Loss: 0.1421, Acc: 95.2243
		 Training: Epoch(4) - Loss: 0.1120, Acc: 96.4228
		 Validation(4) - Loss: 0.1166, Acc: 96.1794
		 Training: Epoch(5) - Loss: 0.0866, Acc: 97.2721
		 Validation(5) - Loss: 0.1132, Acc: 96.5116
		 Training: Epoch(6) - Loss: 0.0832, Acc: 97.3275
		 Validation(6) - Loss: 0.1322, Acc: 95.8056
		 Training: Epoch(7) - Loss: 0.0678, Acc: 97.8075
		 Validation(7) - Loss: 0.1051, Acc: 96.8439
		 Training: Epoch(8) - Loss: 0.0555, Acc: 98.3383
		 Validation(8) - Loss: 0.1103, Acc: 96.9684

		 Training: Epoch(0) - Loss: 1.0792, Acc: 73.9454
		 Validation(0) - Loss: 0.2641, Acc: 92.1064
		 Training: Epoch(1) - Loss: 0.2874, Acc: 91.1105
		 Validation(1) - Loss: 0.1905, Acc: 93.8928
		 Training: Epoch(2) - Loss: 0.1888, Acc: 94.0275
		 Validation(2) - Loss: 0.1555, Acc: 95.2223
		 Training: Epoch(3) - Loss: 0.1389, Acc: 95.5876
		 Validation(3) - Loss: 0.1258, Acc: 96.2609
		 Training: Epoch(4) - Loss: 0.1113, Acc: 96.3537
		 Validation(4) - Loss: 0.1376, Acc: 95.9285
		 Training: Epoch(5) - Loss: 0.0930, Acc: 97.0645
		 Validation(5) - Loss: 0.1220, Acc: 96.5933
		 Training: Epoch(6) - Loss: 0.0816, Acc: 97.3691
		 Validation(6) - Loss: 0.1139, Acc: 96.4271
		 Training: Epoch(7) - Loss: 0.0659, Acc: 97.9692
		 Validation(7) - Loss: 0.1285, Acc: 96.1363
		 Training: Epoch(8) - Loss: 0.0561, Acc: 98.2369
		 Validation(8) - Loss: 0.1664, Acc: 95.6793
		 Training: Epoch(9) - Loss: 0.0482, Acc: 98.5046
		 Validation(9) - Loss: 0.1127, Acc: 96.8010
Finished.
Total time per fold:

In [10]:
print("Train accuracy average: ", np.mean(train_acc) / 100)
print("Top-1 test accuracy average: ", np.mean(test_top1_acc))
print("Top-5 test accuracy average: ", np.mean(test_top5_acc))
print("Weighted Precision test accuracy average: ", np.mean(test_precision))
print("Weighted Recall test accuracy average: ", np.mean(test_recall))
print("Weighted F1 test accuracy average: ", np.mean(test_f1))
print("Average time per fold (seconds):", np.mean(times))

Train accuracy average:  0.940392889008419
Top-1 test accuracy average:  0.9661443919796497
Top-5 test accuracy average:  0.9965106617327368
Weighted Precision test accuracy average:  0.9676727736674009
Weighted Recall test accuracy average:  0.9661443919796497
Weighted F1 test accuracy average:  0.9661999286228173
Average time per fold (seconds): 2963.5463144779205
